In [1]:
#script_dssp
import pandas as pd
import os
import re
import subprocess


In [3]:
#Load the CSV file into a Pandas DataFrame
skempi_file = pd.read_csv('C:\\Users\\Lorenzo\\Desktop\\Tesi\\SKEMPI\\skempi_v2.csv', sep = ';') #it is separeted by a semicolon

# Update this with the path to your SKEMPI file
pdb_dir = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB'  # Your PDB directory

I have to remove the mutations which are not single point

In [4]:


#number of elements in it? (Notice that the position of each element has not changed, so the last element is still 7084)
# len(df_single_mutation)#Create a dataset with only single point mutations to work on

df_single_mutation_skempi = skempi_file[skempi_file['Mutation(s)_PDB'].apply(lambda x: len(x.split(',')) == 1)]

df_single_mutation_skempi.iloc[:, 0:20]

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,Reference,Protein 1,Protein 2,Temperature,kon_mut (M^(-1)s^(-1)),kon_mut_parsed,kon_wt (M^(-1)s^(-1)),kon_wt_parsed,koff_mut (s^(-1)),koff_mut_parsed
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7079,3QIB_ABP_CD,KP9A,KP8A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,NaN,NaN,NaN,NaN,NaN,NaN
7080,3QIB_ABP_CD,KP9R,KP8R,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",2.4E-04,2.400000e-04,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,2070.0,2070.0,4000.0,4000.0,5E-01,0.500
7081,3QIB_ABP_CD,TP12A,TP11A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,NaN,NaN,NaN,NaN,NaN,NaN
7082,3QIB_ABP_CD,TP12S,TP11S,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",3.38E-05,3.380000e-05,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,3960.0,3960.0,4000.0,4000.0,1.34E-01,0.134


#MODELLER



Writing the script to obtain the PDB, chain and mutated amino acid

Use mutation(s)_cleaned 

In [6]:
#Prepare all the information for the script that I will use to mutate the amino acids 

def extract_info(df):
    extracted_info = []
    for index, row in df.iterrows():
        pdb_id = row['#Pdb'].split('_')[0]
        original_aa = row['Mutation(s)_cleaned'][0]
        chain = row['Mutation(s)_cleaned'][1]

        # Use regular expression to find the position (one or more digits)
        position_match = re.search(r'\d+', row['Mutation(s)_cleaned'])
        position = position_match.group() if position_match else None

        mutated_aa = row['Mutation(s)_cleaned'][-1]

        extracted_info.append((pdb_id, original_aa, chain, position, mutated_aa))
    return extracted_info


mutations_info = extract_info(df_single_mutation_skempi)

mutations_info


[('1CSE', 'L', 'I', '38', 'G'),
 ('1CSE', 'L', 'I', '38', 'S'),
 ('1CSE', 'L', 'I', '38', 'P'),
 ('1CSE', 'L', 'I', '38', 'I'),
 ('1CSE', 'L', 'I', '38', 'D'),
 ('1CSE', 'L', 'I', '38', 'E'),
 ('1ACB', 'L', 'I', '38', 'G'),
 ('1ACB', 'L', 'I', '38', 'S'),
 ('1ACB', 'L', 'I', '38', 'P'),
 ('1ACB', 'L', 'I', '38', 'I'),
 ('1ACB', 'L', 'I', '38', 'D'),
 ('1ACB', 'L', 'I', '38', 'E'),
 ('1SBN', 'R', 'I', '38', 'K'),
 ('1SIB', 'K', 'I', '46', 'R'),
 ('1TM1', 'Y', 'I', '42', 'A'),
 ('1TM1', 'Y', 'I', '42', 'G'),
 ('1TM1', 'R', 'I', '46', 'A'),
 ('1TM1', 'R', 'I', '48', 'A'),
 ('1TM1', 'R', 'I', '48', 'C'),
 ('1TM1', 'T', 'I', '39', 'D'),
 ('1TM1', 'T', 'I', '39', 'A'),
 ('1TM1', 'E', 'I', '41', 'A'),
 ('1TM1', 'V', 'I', '51', 'A'),
 ('1Y1K', 'A', 'I', '39', 'T'),
 ('1Y33', 'P', 'I', '39', 'T'),
 ('1Y34', 'A', 'I', '41', 'E'),
 ('1Y3B', 'S', 'I', '41', 'E'),
 ('1Y3C', 'A', 'I', '43', 'R'),
 ('1Y48', 'A', 'I', '46', 'R'),
 ('1Y3D', 'A', 'I', '48', 'R'),
 ('1TM1', 'T', 'I', '39', 'A'),
 ('1TM1'

In [10]:
mutation_list = mutations_info

# Create a set of unique identifiers for each mutation
unique_mutations = set()
for mutation in mutation_list:
    pdb_id, original_aa, chain, position, mutated_aa = mutation
    identifier = f"{pdb_id}_{chain}_{position}_{original_aa}_{mutated_aa}"
    unique_mutations.add(identifier)

# Compare the size of the set to the length of the original list
number_of_unique_mutations = len(unique_mutations)
number_of_total_mutations = len(mutation_list)

# Print the results
print(f"Total number of mutations: {number_of_total_mutations}")
print(f"Number of unique mutations: {number_of_unique_mutations}")
print(f"Number of duplicate mutations: {number_of_total_mutations - number_of_unique_mutations}")


Total number of mutations: 5112
Number of unique mutations: 4337
Number of duplicate mutations: 775


In [16]:
# Finding duplicates
from collections import Counter

# Counting all elements
element_counter = Counter(mutation_list)

# Extracting duplicates (elements with count more than 1)
duplicates = [item for item, count in element_counter.items() if count > 1 and item[0] == '1TM1']

duplicates

[('1TM1', 'Y', 'I', '42', 'A'),
 ('1TM1', 'R', 'I', '46', 'A'),
 ('1TM1', 'R', 'I', '48', 'A'),
 ('1TM1', 'T', 'I', '39', 'A'),
 ('1TM1', 'E', 'I', '41', 'A')]

In [18]:
pdb_start = '1TM1'
# Filter elements where the PDB column starts with the specified string
filtered_elements = df_single_mutation_skempi[df_single_mutation_skempi['#Pdb'].str.startswith(pdb_start)]


filtered_elements

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,koff_mut_parsed,koff_wt (s^(-1)),koff_wt_parsed,dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,Method,SKEMPI version
14,1TM1_E_I,YI61A,YI42A,COR,Pr/PI,Pr/PI,7.92E-11,7.920000e-11,2E-12,2.000000e-12,...,0.000256,1.06E-05,0.000011,NaN,NaN,NaN,NaN,NaN,SFFL,1
15,1TM1_E_I,YI61G,YI42G,COR,Pr/PI,Pr/PI,5.4E-09,5.400000e-09,2E-12,2.000000e-12,...,0.018800,1.06E-05,0.000011,NaN,NaN,NaN,NaN,NaN,SFFL,1
16,1TM1_E_I,RI65A,RI46A,INT,Pr/PI,Pr/PI,3.62E-10,3.620000e-10,2E-12,2.000000e-12,...,0.000347,6.1E-06,0.000006,NaN,NaN,NaN,NaN,NaN,SFFL,1
17,1TM1_E_I,RI67A,RI48A,SUP,Pr/PI,Pr/PI,2.78E-10,2.780000e-10,2E-12,2.000000e-12,...,0.000315,6.1E-06,0.000006,NaN,NaN,NaN,NaN,NaN,SFFL,1
18,1TM1_E_I,RI67C,RI48C,SUP,Pr/PI,Pr/PI,4.71E-10,4.710000e-10,2E-12,2.000000e-12,...,0.000303,6.1E-06,0.000006,NaN,NaN,NaN,NaN,NaN,SFFL,1
20,1TM1_E_I,TI58D,TI39D,COR,Pr/PI,Pr/PI,2.42E-11,2.420000e-11,7E-13,7.000000e-13,...,0.000034,3.9E-06,0.000004,NaN,NaN,NaN,NaN,NaN,IASP,1
21,1TM1_E_I,TI58A,TI39A,COR,Pr/PI,Pr/PI,5.38E-11,5.380000e-11,7E-13,7.000000e-13,...,0.000206,3.9E-06,0.000004,NaN,NaN,NaN,NaN,NaN,IASP,1
22,1TM1_E_I,EI60A,EI41A,COR,Pr/PI,Pr/PI,9.75E-11,9.750000e-11,7E-13,7.000000e-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
25,1TM1_E_I,VI70A,VI51A,INT,Pr/PI,Pr/PI,7.3E-13,7.300000e-13,7E-13,7.000000e-13,...,0.000003,3.9E-06,0.000004,NaN,NaN,NaN,NaN,NaN,IASP,1
34,1TM1_E_I,TI58A,TI39A,COR,Pr/PI,Pr/PI,3E-10,3.000000e-10,3E-12,3.000000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1


In [19]:
two_mutations_info = [('1CSE', 'L', 'I', '38', 'G'),
                       ('1CSE', 'L', 'I', '38', 'S')]

The following script uses MODELLER. I would use this if you have to start from scratch

In [ ]:
import subprocess
import os

# Define the paths
modeler_script_path = r'C:/Users/Lorenzo/Desktop/Tesi/Python_code/modeler_script.py'
input_directory = r'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB/'  # Ensure trailing slash
output_directory = r'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/'  # Ensure trailing slash

# Dictionary to track occurrences of each mutation
mutation_occurrences = {}

# Iterate over the extracted information
for pdb_id, original_aa, chain, position, mutated_aa in mutations_info:
    mutation_str = original_aa + position + mutated_aa
    pdb_file = os.path.join(input_directory, pdb_id + '.pdb')

    # Construct the base file name
    base_file_name = f"{pdb_id}_{original_aa}{chain}{position}{mutated_aa}"

    # Update the occurrence count
    mutation_occurrences[base_file_name] = mutation_occurrences.get(base_file_name, 0) + 1
    duplicate_count = mutation_occurrences[base_file_name] - 1

    # Construct the file name, add "_duplicate_" suffix if not the first occurrence
    if duplicate_count > 0:
        new_file_name = os.path.join(output_directory, f"{base_file_name}_duplicate_{duplicate_count}.pdb")
    else:
        new_file_name = os.path.join(output_directory, base_file_name + '.pdb')

    # Check if the file name already exists
    if os.path.exists(new_file_name):
        continue  # Skip this iteration if file already exists

    # Construct the command as a list
    command = ['python', modeler_script_path, pdb_file, chain, mutation_str, '-n', '1', '-o', new_file_name, '-v']

    print("Executing command:", ' '.join(command))
    # Execute the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)

    # Print any output or error
    print("Output:", result.stdout)
    print("Error:", result.stderr)


The following two scripts have a problem. The first does not create duplicates, the second is made only if you already used the first.

In [6]:
# Define the paths (using raw strings for Windows paths)
modeler_script_path = r'C:/Users/Lorenzo/Desktop/Tesi/Python_code/modeler_script.py'
input_directory = r'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB/'  # Ensure trailing slash
output_directory = r'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/'  # Ensure trailing slash

# Iterate over the extracted information
for pdb_id, original_aa, chain, position, mutated_aa in mutations_info:
    mutation_str = original_aa + position + mutated_aa
    pdb_file = input_directory + pdb_id + '.pdb'
    new_file_name = output_directory + f"{pdb_id}_{original_aa}{chain}{position}{mutated_aa}.pdb"

    # Construct the command as a list
    command = ['python', modeler_script_path, pdb_file, chain, mutation_str, '-n', '1', '-o', new_file_name, '-v']

    # Check if the output file already exists
    if os.path.exists(new_file_name):
        print(f"File {new_file_name} already exists. Skipping...")
        continue

    print("Executing command:", ' '.join(command))
    # Execute the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)

    # Print any output or error
    print("Output:", result.stdout)
    print("Error:", result.stderr)

File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1CSE_LI38G.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1CSE_LI38S.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1CSE_LI38P.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1CSE_LI38I.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1CSE_LI38D.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1CSE_LI38E.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1ACB_LI38G.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1ACB_LI38S.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1ACB_LI38P.pdb already exists. Skipping...
File C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/1ACB_LI38I.pdb already 

In [7]:
import subprocess
import os

# Define the paths
modeler_script_path = r'C:/Users/Lorenzo/Desktop/Tesi/Python_code/modeler_script.py'
input_directory = r'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB/'  # Ensure trailing slash
output_directory = r'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/'  # Ensure trailing slash

# Dictionary to track occurrences of each mutation
mutation_occurrences = {}

# Iterate over the extracted information
for pdb_id, original_aa, chain, position, mutated_aa in mutations_info:
    mutation_str = original_aa + position + mutated_aa
    pdb_file = os.path.join(input_directory, pdb_id + '.pdb')

    # Construct the base file name
    base_file_name = f"{pdb_id}_{original_aa}{chain}{position}{mutated_aa}"

    # Update the occurrence count
    mutation_occurrences[base_file_name] = mutation_occurrences.get(base_file_name, 0) + 1

    # Skip the first occurrence
    if mutation_occurrences[base_file_name] == 1:
        continue

    # Update the file name for second and subsequent occurrences
    duplicate_count = mutation_occurrences[base_file_name] - 1
    new_file_name = os.path.join(output_directory, f"{base_file_name}_duplicate_{duplicate_count}.pdb")

    # Check if the duplicate file name already exists
    if os.path.exists(new_file_name):
        continue  # Skip this iteration if file already exists

    # Construct the command as a list
    command = ['python', modeler_script_path, pdb_file, chain, mutation_str, '-n', '1', '-o', new_file_name, '-v']

    print("Executing command:", ' '.join(command))
    # Execute the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)

    # Print any output or error
    print("Output:", result.stdout)
    print("Error:", result.stderr)



Executing command: python C:/Users/Lorenzo/Desktop/Tesi/Python_code/modeler_script.py C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB/3S9D.pdb A L8A -n 1 -o C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutanti_modeller/3S9D_LA8A_duplicate_1.pdb -v
Output: 
                         MODELLER 10.4, 2022/10/28, r12463

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2022 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, 

This is the command that modeler_script.py accepts:

python modeler_script.py protein.pdb A A23T,B45G -o mutated_protein.pdb -v

#Extracting DSSP 

In [4]:
def extract_monomers(skempi_file, pdb_dir):
    
    for index, row in skempi_file.iterrows():
        pdb_id = row['#Pdb'].split('_')[0]
        chain_groups = row['#Pdb'].split('_')[1:]

        pdb_file_path = os.path.join(pdb_dir, f'{pdb_id}.pdb')

        if os.path.exists(pdb_file_path):
            with open(pdb_file_path, 'r') as file:
                pdb_content = file.readlines()

            for chain_group in chain_groups:
                for chain in chain_group:
                    # Filter lines corresponding to the specific chain
                    monomer_content = [line for line in pdb_content if line.startswith('ATOM') and line[21] == chain]

                    # Define the output file name including the chain identifier
                    monomer_file_path = os.path.join(pdb_dir, f'{pdb_id}_{chain}.pdb')

                    with open(monomer_file_path, 'w') as monomer_file:
                        monomer_file.writelines(monomer_content)
                        print(f'Monomer file created: {monomer_file_path}')
        else:
            print(f'PDB file not found for {pdb_id}')

# Run the function
extract_monomers(df_single_mutation_skempi, pdb_dir)

Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_I.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_I.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_I.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_I.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_I.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_I.pdb
Monomer file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E.pdb
Monomer file created: C:/Users/Lorenzo

Now I need another script that creates a new file that contains both chains that form the complex. The only difference is that it creates a unique file for each skempi file

In [6]:
def extract_complexes(skempi_file, pdb_dir):
    
    for index, row in skempi_file.iterrows():
        pdb_id = row['#Pdb'].split('_')[0]
        chain_groups = row['#Pdb'].split('_')[1:]
        all_chains = ''.join(chain_groups)

        pdb_file_path = os.path.join(pdb_dir, f'{pdb_id}.pdb')

        if os.path.exists(pdb_file_path):
            with open(pdb_file_path, 'r') as file:
                pdb_content = file.readlines()

            # Filter lines corresponding to the chains in all chain groups
            complex_content = [line for line in pdb_content if line.startswith('ATOM') and line[21] in all_chains]

            # Define the output file name including all chain identifiers
            complex_file_path = os.path.join(pdb_dir, f'{pdb_id}_{"_".join(chain_groups)}.pdb')

            with open(complex_file_path, 'w') as complex_file:
                complex_file.writelines(complex_content)
                print(f'Complex file created: {complex_file_path}')
        else:
            print(f'PDB file not found for {pdb_id}')

# Run the function
extract_complexes(df_single_mutation_skempi, pdb_dir)

Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1SBN_E_I.pdb
Complex file

In [6]:
#Old function, it has to be deleted

def extract_complexes(skempi_df, pdb_dir):
    for index, row in skempi_df.iterrows():
        pdb_id = row['#Pdb'].split('_')[0]
        chain_groups = row['#Pdb'].split('_')[1:]

        pdb_file_path = os.path.join(pdb_dir, f'{pdb_id}.pdb')

        if os.path.exists(pdb_file_path):
            with open(pdb_file_path, 'r') as file:
                pdb_content = file.readlines()

            complex_content = []
            for group in chain_groups:
                for chain in group:
                    # Filter lines corresponding to the specific chain
                    chain_content = [line for line in pdb_content if line.startswith('ATOM') and line[21] == chain]
                    complex_content.extend(chain_content)

            # Define the output file name for the complex
            complex_file_path = os.path.join(pdb_dir, f'{pdb_id}_{"_".join(chain_groups)}.pdb')

            with open(complex_file_path, 'w') as complex_file:
                complex_file.writelines(complex_content)
                print(f'Complex file created: {complex_file_path}')
        else:
            print(f'PDB file not found for {pdb_id}')

# Run the function
extract_complexes(df_single_mutation_skempi, pdb_dir)


Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1CSE_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1ACB_E_I.pdb
Complex file created: C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB\1SBN_E_I.pdb
Complex file

In [8]:
pip install biopython


     ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
     --- ------------------------------------ 0.2/2.7 MB 4.1 MB/s eta 0:00:01
     ----------- ---------------------------- 0.8/2.7 MB 8.3 MB/s eta 0:00:01
     ---------------------- ----------------- 1.5/2.7 MB 10.9 MB/s eta 0:00:01
     ----------------------- ---------------- 1.6/2.7 MB 9.9 MB/s eta 0:00:01
     ----------------------- ---------------- 1.6/2.7 MB 9.9 MB/s eta 0:00:01
     --------------------------------- ------ 2.3/2.7 MB 8.0 MB/s eta 0:00:01
     ---------------------------------------  2.7/2.7 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------- 2.7/2.7 MB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\Lorenzo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


#Calculate DSSP

In [14]:
import os
from Bio.PDB import PDBParser, DSSP

def calculate_dssp(pdb_folder, output_folder):
    parser = PDBParser(QUIET=True)  # QUIET=True suppresses warnings
    for filename in os.listdir(pdb_folder):
        if filename.endswith(".pdb"):
            pdb_path = os.path.join(pdb_folder, filename)
            structure = parser.get_structure(filename, pdb_path)

            # Calculate DSSP
            model = structure[0]
            try:
                dssp = DSSP(model, pdb_path)  # Assuming dssp is in PATH
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue

            # Output file path
            output_file_path = os.path.join(output_folder, filename + "_dssp.txt")

            # Write the DSSP results for each residue to a file
            with open(output_file_path, 'w') as file:
                for residue in dssp:
                    file.write(f"Residue: {residue[0]}, Secondary Structure: {residue[2]}\n")

# Define the directory and output directory
pdb_dir = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_prova'
output_dir = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp_mutant_complexes'

# Run the function
calculate_dssp(pdb_dir, output_dir)



Error processing 1A4Y.pdb: [WinError 2] The system cannot find the file specified
Error processing 1CSE.pdb: [WinError 2] The system cannot find the file specified


In [1]:
import os
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

In [2]:
def calculate_dssp(pdb_folder, dssp_executable_path, output_folder):
    parser = PDBParser()
    for filename in os.listdir(pdb_folder):
        if filename.endswith(".pdb"):
            pdb_path = os.path.join(pdb_folder, filename)
            structure = parser.get_structure(filename, pdb_path)

            # Calculate DSSP
            model = structure[0]
            dssp = DSSP(model, pdb_path, dssp_executable=dssp_executable_path)

            # Prepare output file path
            output_file_path = os.path.join(output_folder, filename + '_dssp.txt')

            # Write the DSSP results to a file
            with open(output_file_path, 'w') as file:
                for residue in dssp:
                    file.write(f"Residue: {residue[0]}, Secondary Structure: {residue[2]}\n")

# Define the directories and DSSP executable path
pdb_dir = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_prova/'
dssp_path = '/usr/bin/dssp'  # Update this with the path to your DSSP executable if not in PATH
output_dir = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/dssp/dssp_prova'  # Output directory

# Run the function
calculate_dssp(pdb_dir, dssp_path, output_dir)


C:\Users\Lorenzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 3411
  warnings.warn(
C:\Users\Lorenzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 4405
  warnings.warn(
C:\Users\Lorenzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4406.
  warnings.warn(
C:\Users\Lorenzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontin

TypeError: DSSP.__init__() got an unexpected keyword argument 'dssp_executable'

In [7]:
pip install biopython


Calculate the distances and contact of amino acids

In [9]:
from Bio.PDB import PDBParser
import pandas as pd
import numpy as np
import os

In [16]:
#Load the CSV file into a Pandas DataFrame
skempi_file = pd.read_csv('C:\\Users\\Lorenzo\\Desktop\\Tesi\\SKEMPI\\skempi_v2.csv', sep = ';') #it is separeted by a semicolon
# Update this with the path to your SKEMPI file
pdb_dir = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI'  # Your PDB directory
# Define the folder containing PDB files
pdb_folder_path = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB'  # Update with the path to your folder containing PDB files
# Define the distance threshold (in Ångströms)
threshold = 5

def parse_mutation(mutation_str):
    """
    Parse the mutation string to extract:
    - The original amino acid
    - The chain identifier
    - The residue number (and insertion code if applicable)
    - The mutated amino acid
    Handles formats like 'LI38G'.
    """
    original_aa = mutation_str[0]  # First character: original amino acid
    chain_id = mutation_str[1]     # Second character: chain identifier
    mutated_aa = mutation_str[-1]  # Last character: mutated amino acid

    # Extracting residue number and insertion code (if present)
    residue_info = mutation_str[2:-1]
    residue_number = ''.join(filter(str.isdigit, residue_info))
    insertion_code = ''.join(filter(str.isalpha, residue_info))

    # Convert the residue number to an integer
    residue_number = int(residue_number) if residue_number else None

    return original_aa, chain_id, residue_number, insertion_code, mutated_aa


def calc_distance(atom1, atom2):
    diff_vector = atom1.coord - atom2.coord
    return np.sqrt(np.sum(diff_vector * diff_vector))
# Open a text file to write the results
with open('C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/Results.txt', 'w') as result_file:
    # PDB Parser
    parser = PDBParser()
    # Process each PDB file in the folder
    for pdb_file in os.listdir(pdb_folder_path):
        if pdb_file.endswith('.pdb'):
            pdb_file_path = os.path.join(pdb_folder_path, pdb_file)
            structure = parser.get_structure('PDB', pdb_file_path)

            # Assuming the first chain in the structure is the relevant one
            chain = structure[0][list(structure[0].get_chains())[0].id]

            # Process each mutation in the SKEMPI dataset relevant to this PDB file
            distance_results = []
            for index, row in skempi_file.iterrows():
                # Implement logic to match the mutation to the PDB file
                # Example: if pdb_file matches with row['PDB_ID'] or some other field
                mutation_str = row['Mutation(s)_cleaned']
                original_aa, chain_id, residue_number, insertion_code, mutated_aa = parse_mutation(mutation_str)
                

                if residue_number in chain:
                    mutated_residue = chain[residue_number]
                    if 'CA' in mutated_residue:
                        ca_mutated = mutated_residue['CA']
                        for res in chain:
                            if 'CA' in res and res != mutated_residue:
                                distance = calc_distance(ca_mutated, res['CA'])
                                if distance <= threshold:
                                    distance_results.append((mutation_str, res.get_resname(), res.id[1], distance))
            # Write results for this PDB file to the file
            result_file.write(f"Results for PDB file: {pdb_file}\n")
            for result in distance_results:
                result_file.write(f"Mutation: {result[0]}, Residue: {result[1]}, Position: {result[2]}, Distance: {result[3]:.2f} Å\n")
            result_file.write("\n")

            '''
            # Output results for this PDB file
            print(f"Results for PDB file: {pdb_file}")
            for result in distance_results:
                print(f"Mutation: {result[0]}, Residue: {result[1]}, Position: {result[2]}, Distance: {result[3]:.2f} Å")
            print("\n")
            '''

c:\Users\Lorenzo\Anacond_bioinformatics\lib\site-packages\Bio\PDB\PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 1467
  warnings.warn(
c:\Users\Lorenzo\Anacond_bioinformatics\lib\site-packages\Bio\PDB\PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 3046
  warnings.warn(
c:\Users\Lorenzo\Anacond_bioinformatics\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3047.
  warnings.warn(
c:\Users\Lorenzo\Anacond_bioinformatics\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3073.
  warnings.warn(
c:\Users\Lorenzo\Anacond_bioinformatics\lib\site-packages\Bio\PDB\PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 3411
  warnings.warn(
c:\Users\Lorenzo\Anacond_bioinformatics\lib\site-packages\Bio\PDB\PDBParser.py:395: PDBConstructionWarning: Ignoring unreco

In [8]:
from Bio.PDB import PDBParser
import pandas as pd
import os

In [9]:
df_single_mutation_skempi.iloc[:, 0:20]

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,Reference,Protein 1,Protein 2,Temperature,kon_mut (M^(-1)s^(-1)),kon_mut_parsed,kon_wt (M^(-1)s^(-1)),kon_wt_parsed,koff_mut (s^(-1)),koff_mut_parsed
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,9048543,Subtilisin Carlsberg,Eglin c,294,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7079,3QIB_ABP_CD,KP9A,KP8A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,NaN,NaN,NaN,NaN,NaN,NaN
7080,3QIB_ABP_CD,KP9R,KP8R,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",2.4E-04,2.400000e-04,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,2070.0,2070.0,4000.0,4000.0,5E-01,0.500
7081,3QIB_ABP_CD,TP12A,TP11A,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",>1.1E-03,1.100000e-03,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,NaN,NaN,NaN,NaN,NaN,NaN
7082,3QIB_ABP_CD,TP12S,TP11S,COR,TCR/pMHC,"TCR/pMHC,1JCK_A_B",3.38E-05,3.380000e-05,5.5E-06,5.500000e-06,12152083,I-Ek plus MCC peptide,2B4 TCR,298,3960.0,3960.0,4000.0,4000.0,1.34E-01,0.134


Find amino acid contacts
For the mutated PDBs is necessary a correction in the code in order to take the mutated amino acid from the SKEMPI dataset, not the original one

Running this code produced 4337 txt files and not 5112 files as one would expect. This is because some mutations are the same, but obtained with different experiments. In this case it does not matter, since we are counting the contacts of the original amino acids, not the mutated ones. Therefore in both cases we obtain the same txt file.

In [12]:
import os
import pandas as pd
from Bio.PDB import PDBParser, NeighborSearch
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning

# Suppress PDBConstructionWarning warnings
warnings.simplefilter('ignore', PDBConstructionWarning)

def parse_amino_acid_info(amino_acid_info):
    # Extracting chain ID (second letter) and residue position
    chain_id = amino_acid_info[1]  # Second character is the chain ID
    residue_pos = int(''.join(filter(str.isdigit, amino_acid_info)))  # Extracting numerical part
    return chain_id, residue_pos

def find_amino_acid_info(df, pdb_id):
    entries = df[df['#Pdb'].str.contains(pdb_id, case=False)]
    return entries['Mutation(s)_cleaned'].tolist() if not entries.empty else []

def calculate_distances(pdb_directory, pdb_file, amino_acid_info):
    try:
        parser = PDBParser()
        structure = parser.get_structure(pdb_file, os.path.join(pdb_directory, f"{pdb_file}.pdb"))
        model = structure[0]

        contacts = set()
        for mutation_info in amino_acid_info:
            chain_id, residue_pos = parse_amino_acid_info(mutation_info)

            if chain_id not in model:
                print(f"Chain {chain_id} not found in {pdb_file}")
                continue
            if residue_pos not in model[chain_id]:
                print(f"Residue position {residue_pos} not found in chain {chain_id} of {pdb_file}")
                continue

            specified_residue = model[chain_id][residue_pos]
            atoms = [atom for atom in model.get_atoms()]
            ns = NeighborSearch(atoms)

            for atom in specified_residue:
                for nearby_atom in ns.search(atom.coord, 5.0, 'A'):
                    if nearby_atom.parent != specified_residue and nearby_atom.parent.resname != "HOH":
                        contacts.add((nearby_atom.parent.resname, nearby_atom.parent.id[1], nearby_atom.parent.parent.id))

        return list(contacts)
    except Exception as e:
        print(f"Error processing {pdb_file} for amino acid {amino_acid_info}: {e}")
        return []

def write_output_file(pdb_file, amino_acid_info, contacts):
    output_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/original_aminoacid_contacts'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Sanitize amino_acid_info for use in filename
    valid_amino_acid_info = "".join(x for x in amino_acid_info if x.isalnum())
    output_filename = os.path.join(output_directory, f"{pdb_file}_{valid_amino_acid_info}_contacts.txt")

    with open(output_filename, 'w') as file:
        file.write(f"PDB File: {pdb_file}\n")
        file.write(f"Amino Acid Info: {amino_acid_info}\n")
        file.write("Contacts with other amino acids:\n")
        for resname, pos, chain_id in contacts:
            file.write(f"{resname} {pos} {chain_id}\n")

# Main execution
pdb_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_original'
pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]

for pdb_file in pdb_files:
    pdb_id = pdb_file.split('.')[0]
    amino_acid_infos = find_amino_acid_info(df_single_mutation_skempi, pdb_id)
    if amino_acid_infos:
        for amino_acid_info in amino_acid_infos:
            contacts = calculate_distances(pdb_directory, pdb_id, [amino_acid_info])
            write_output_file(pdb_id, amino_acid_info, contacts)
    else:
        print(f"Amino acid information not found in the dataset for {pdb_id}.")


Amino acid information not found in the dataset for 1MQ8.
Amino acid information not found in the dataset for 1OHZ.
Amino acid information not found in the dataset for 1QAB.
Amino acid information not found in the dataset for 1XXM.
Amino acid information not found in the dataset for 1Y4A.
Amino acid information not found in the dataset for 2ABZ.
Amino acid information not found in the dataset for 2CCL.
Amino acid information not found in the dataset for 2NOJ.
Amino acid information not found in the dataset for 2NY7.
Amino acid information not found in the dataset for 2PYE.
Amino acid information not found in the dataset for 2QJ9.
Amino acid information not found in the dataset for 2QJA.
Amino acid information not found in the dataset for 2QJB.
Amino acid information not found in the dataset for 2VN5.
Amino acid information not found in the dataset for 3BDY.
Amino acid information not found in the dataset for 3BE1.
Amino acid information not found in the dataset for 3BIW.
Amino acid inf

Prova

In [7]:
import os
import pandas as pd
from Bio.PDB import PDBParser, NeighborSearch
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning

# Suppress PDBConstructionWarning warnings
warnings.simplefilter('ignore', PDBConstructionWarning)

def parse_amino_acid_info(amino_acid_info):
    # Extracting chain ID (second letter) and residue position
    chain_id = amino_acid_info[1]  # Second character is the chain ID
    residue_pos = int(''.join(filter(str.isdigit, amino_acid_info)))  # Extracting numerical part
    return chain_id, residue_pos

def find_amino_acid_info(df, pdb_id):
    entries = df[df['#Pdb'].str.contains(pdb_id, case=False)]
    return entries['Mutation(s)_cleaned'].tolist() if not entries.empty else []

def calculate_distances(pdb_directory, pdb_file, amino_acid_info):
    try:
        parser = PDBParser()
        structure = parser.get_structure(pdb_file, os.path.join(pdb_directory, f"{pdb_file}.pdb"))
        model = structure[0]

        contacts = set()
        for mutation_info in amino_acid_info:
            chain_id, residue_pos = parse_amino_acid_info(mutation_info)

            if chain_id not in model:
                print(f"Chain {chain_id} not found in {pdb_file}")
                continue
            if residue_pos not in model[chain_id]:
                print(f"Residue position {residue_pos} not found in chain {chain_id} of {pdb_file}")
                continue

            specified_residue = model[chain_id][residue_pos]
            atoms = [atom for atom in model.get_atoms()]
            ns = NeighborSearch(atoms)

            for atom in specified_residue:
                for nearby_atom in ns.search(atom.coord, 5.0, 'A'):
                    if nearby_atom.parent != specified_residue and nearby_atom.parent.resname != "HOH":
                        contacts.add((nearby_atom.parent.resname, nearby_atom.parent.id[1], nearby_atom.parent.parent.id))

        return list(contacts)
    except Exception as e:
        print(f"Error processing {pdb_file} for amino acid {amino_acid_info}: {e}")
        return []

def write_output_file(pdb_file, amino_acid_info, contacts):
    output_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/test_directory'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Sanitize amino_acid_info for use in filename
    valid_amino_acid_info = "".join(x for x in amino_acid_info if x.isalnum())
    output_filename = os.path.join(output_directory, f"{pdb_file}_{valid_amino_acid_info}_contacts.txt")

    with open(output_filename, 'w') as file:
        file.write(f"PDB File: {pdb_file}\n")
        file.write(f"Amino Acid Info: {amino_acid_info}\n")
        file.write("Contacts with other amino acids:\n")
        for resname, pos, chain_id in contacts:
            file.write(f"{resname} {pos} {chain_id}\n")

# Main execution
pdb_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_test'
pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]

for pdb_file in pdb_files:
    pdb_id = pdb_file.split('.')[0]
    amino_acid_infos = find_amino_acid_info(df_single_mutation_skempi, pdb_id)
    if amino_acid_infos:
        for amino_acid_info in amino_acid_infos:
            contacts = calculate_distances(pdb_directory, pdb_id, [amino_acid_info])
            write_output_file(pdb_id, amino_acid_info, contacts)
    else:
        print(f"Amino acid information not found in the dataset for {pdb_id}.")

['MA14A']
['HA18A']
['HA21A']
['QA22A']
['FA25A']
['DA26A']
['QA29A']
['YA42A']
['LA45A']
['QA46A']
['PA48A']
['SA51A']
['EA56A']
['PA61A']
['SA62A']
['NA63A']
['RA64A']
['EA65A']
['QA68A']
['YA153A']
['RA156A']
['KA157A']
['DA160A']
['KA161A']
['EA163A']
['TA164A']
['FA165A']
['RA167A']
['IA168A']
['RA172A']
['EA175A']
['RB11A']
['EB12A']
['RB29A']
['RB30A']
['TB32A']
['QB33A']
['EB34A']
['WB35A']
['WB39A']
['SB57A']
['SB61A']
['IB62A']
['WB63A']
['IB64A']
['PB65A']
['EB79A']
['KB80A']
['SB83A']
['DB85A']
['EB86A']
['DB119A']
['IB120A']
['QB121A']
['KB122A']
['WB124A']
['VB126A']
['TB150A']
['QB171A']
['RB172A']
['NB173A']
['SB174A']
['EB10A']
['NB31A']
['TB36A']
['FB55S']
['TB60A']
['TB149A']
['KB170A']
['RB11L']
['RB11M']
['FB55S']
['RB7A']
['EB10A']
['RB11A']
['EB12A']
['RB29A']
['RB30A']
['EB34A']
['EB38A']
['KB40A']
['EB41A']
['EB50A']
['KB69A']
['DB85A']
['EB86A']
['DB91A']
['EB128A']
['EB130A']
['KB134A']
['EB135A']
['KB158A']
['DB160A']
['KB161A']
['EB162A']
['EB164A']
['RB166

Application of the modified code to the mutants 


In [10]:
import os
import pandas as pd
from Bio.PDB import PDBParser, NeighborSearch
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning

# Suppress PDBConstructionWarning warnings
warnings.simplefilter('ignore', PDBConstructionWarning)

def parse_file_name(file_name):
    # Extracting the mutation information from the file name
    parts = file_name.split('_')
    mutation_info = parts[1]  # Assumes the format like 1A22_DA160A
    chain_id = mutation_info[1]  # Second character is the chain ID
    amino_acid = mutation_info[-1]  # Last character is the mutated amino acid
    residue_pos = int(''.join(filter(str.isdigit, mutation_info)))  # Extracting numerical part
    return chain_id, residue_pos, amino_acid

def calculate_distances(pdb_directory, pdb_file, chain_id, residue_pos):
    try:
        parser = PDBParser()
        structure = parser.get_structure(pdb_file, os.path.join(pdb_directory, pdb_file))
        model = structure[0]

        contacts = set()
        specified_residue = model[chain_id][residue_pos]
        atoms = [atom for atom in model.get_atoms()]
        ns = NeighborSearch(atoms)

        for atom in specified_residue:
            for nearby_atom in ns.search(atom.coord, 5.0, 'A'):
                if nearby_atom.parent != specified_residue and nearby_atom.parent.resname != "HOH":
                    contacts.add((nearby_atom.parent.resname, nearby_atom.parent.id[1], nearby_atom.parent.parent.id))

        return list(contacts)
    except Exception as e:
        print(f"Error processing {pdb_file}: {e}")
        return []

def write_output_file(file_name, chain_id, residue_pos, amino_acid, contacts):
    output_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/mutant_aminoacid_contacts'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    mutation_part = f"{chain_id}{residue_pos}{amino_acid}"  # Mutation information
    output_filename = os.path.join(output_directory, f"{file_name}_{mutation_part}_contacts.txt")
    with open(output_filename, 'w') as file:
        file.write(f"PDB File: {file_name}.pdb\n")
        file.write(f"Mutation Info: {mutation_part}\n")
        file.write("Contacts with other amino acids:\n")
        for resname, pos, chain_id in contacts:
            file.write(f"{resname} {pos} {chain_id}\n")

# Main execution
pdb_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller'
pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]

for pdb_file in pdb_files:
    file_name = os.path.splitext(pdb_file)[0]  # Remove the .pdb extension
    chain_id, residue_pos, amino_acid = parse_file_name(file_name)
    contacts = calculate_distances(pdb_directory, pdb_file, chain_id, residue_pos)
    write_output_file(file_name, chain_id, residue_pos, amino_acid, contacts)


Extract chains from mutants of PDB

In [1]:
import os

In [5]:

def extract_chain_from_pdb(file_path, chain_letter):
    """ Extracts atoms from a specified chain in a PDB file and returns them as a string. """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    chain_lines = [line for line in lines if line.startswith("ATOM") and line[21] == chain_letter]
    return ''.join(chain_lines)

def get_chain_letter(filename):
    """ Extracts the chain letter from the filename. It's the second letter after the underscore. """
    parts = filename.split('_')
    if len(parts) > 1 and len(parts[1]) > 1:
        chain_part = parts[1]
        return chain_part[1]  # Getting the second letter
    return None


def process_pdb_files(input_directory, output_directory):
    """ Processes all PDB files in a directory, extracting specified chains and saves them in a different directory. """
    for filename in os.listdir(input_directory):
        if filename.endswith(".pdb"):
            chain_letter = get_chain_letter(filename)
            if chain_letter:
                input_file_path = os.path.join(input_directory, filename)
                chain_data = extract_chain_from_pdb(input_file_path, chain_letter)

                new_filename = f"{filename.split('.')[0]}_{chain_letter}_chain.pdb"
                output_file_path = os.path.join(output_directory, new_filename)

                with open(output_file_path, 'w') as new_file:
                    new_file.write(chain_data)
                print(f"Processed {filename}: Extracted chain {chain_letter} and saved to {output_directory}")

# Example usage
input_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller'  # Replace with the path to your input directory
output_directory = 'C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers'  # Replace with the path to your output directory
process_pdb_files(input_directory, output_directory)


Processed 1A22_CA171A.pdb: Extracted chain A and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_CB67A.pdb: Extracted chain B and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_CB81A.pdb: Extracted chain B and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_DA160A.pdb: Extracted chain A and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_DA160A_duplicate_1.pdb: Extracted chain A and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_DA26A.pdb: Extracted chain A and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_DB119A.pdb: Extracted chain B and saved to C:/Users/Lorenzo/Desktop/Tesi/SKEMPI/PDB_all/PDB_mutanti_modeller_monomers
Processed 1A22_DB119A_duplicate_1.pdb: Extracted chain B and